In [8]:
import pandas as pd 
import numpy as np
from dash_website import DIMENSIONS, MAIN_CATEGORIES_TO_CATEGORIES
from dash_website.utils.graphs.dendrogram_heatmap import create_dendrogram_heatmap
import plotly.graph_objs as go

dimension_1 = "MainDimensions"
subset_method = "union"
correlation_type = "pearson"
main_category = "All"
display_mode = "view_per_main_category"

data_correlations = pd.read_feather(f"../../data/xwas/univariate_correlations/correlations/correlations_{subset_method}_{correlation_type}.feather").to_dict()
data_averages = pd.read_feather("../../data/xwas/univariate_correlations/averages_correlations.feather").to_dict()

averages = pd.DataFrame(data_averages).set_index(["dimension", "category"])
averages.columns = pd.MultiIndex.from_tuples(
    list(map(eval, averages.columns.tolist())), names=["subset_method", "correlation_type", "observation"]
)

In [9]:
selected_categories = ["Phenotypic", "Genetics", "FamilyHistory"] + [f"All_{main_category}" for main_category in MAIN_CATEGORIES_TO_CATEGORIES.keys()]
dict_label = {"Phenotypic": "Phenotypic", "Genetics": "Genetics", "FamilyHistory": "Family History", "All_All": "All", "All_Biomarkers": "Biomarkers", "All_Phenotypes": "Clinical Phenotypes", "All_Diseases": "Diseases", "All_Environmental": "Environmental variables", "All_Socioeconomics": "Socioeconomics"}

In [10]:
sorted_averages = averages.loc[
    (["MainDimensions", "SubDimensions"], selected_categories), (subset_method, correlation_type)].sort_values(by=["mean"], ascending=False)
main_dimension_bar = go.Bar(
    x=sorted_averages.loc["MainDimensions"].rename(index=dict_label).index.get_level_values("category"),
    y=sorted_averages.loc["MainDimensions", "mean"],
    error_y={"array": sorted_averages.loc["MainDimensions", "std"], "type": "data"},
    name="Aging main dimensions",
    marker_color="rgb(26, 118, 255)",
)
subdimension_bar = go.Bar(
    x=sorted_averages.loc["SubDimensions"].rename(index=dict_label).index.get_level_values("category"),
    y=sorted_averages.loc["SubDimensions", "mean"],
    error_y={"array": sorted_averages.loc["SubDimensions", "std"], "type": "data"},
    name="Aging subdimensions",
    marker_color="rgb(55, 83, 109)",
)

fig = go.Figure()
fig.add_trace(main_dimension_bar)
fig.add_trace(subdimension_bar)

fig.update_layout(
    {
        "width": 1000,
        "height": 500,
        "xaxis": {"tickangle": 90, "showgrid": False},
        "yaxis": {"title": "Average correlation"},
    }
)
fig.write_html("overall_correlations.html")

fig.show()